<a href="https://colab.research.google.com/github/ricky850/Test/blob/master/Option.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1DNg3BaB2IJHkJPyYOJ2IK0nUF8U5Yzex'  #雲端硬碟檔案連結碼d/
downloaded = drive.CreateFile({'id' : file_id})
downloaded.GetContentFile('weeklyovsv.csv')
df = pd.DataFrame(pd.read_csv('weeklyovsv.csv')) #index_col='Date'
df.info()
print("original data")
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13182265 entries, 0 to 13182264
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Unnamed: 0      int64  
 1   weeklycallvol   float64
 2   secid           int64  
 3   weeklyputvol    float64
 4   weeklycallnum   float64
 5   weeklyputnum    float64
 6   weeklystockvol  float64
 7   date            int64  
 8   weeklyvol       float64
dtypes: float64(6), int64(3)
memory usage: 905.2 MB
original data
          Unnamed: 0  weeklycallvol  ...      date  weeklyvol
0                  0         5000.0  ...  19960111   0.438961
1                  3         3000.0  ...  19960111   0.090631
2                  5         6100.0  ...  19960111   0.097943
3                  8         3100.0  ...  19960111   0.720609
4                 10       324400.0  ...  19960111   0.119213
...              ...            ...  ...       ...        ...
13182260        3084         8000.0  ...  20171229   0.019806
13182

In [10]:
df.tail(50)

,Unnamed: 0,weeklycallvol,secid,weeklyputvol,weeklycallnum,weeklyputnum,weeklystockvol,date,weeklyvol
13182215,3014,3100.0,207526,1300.0,90.0,90.0,5558.81,20171229,0.130538
13182216,3016,1000.0,207540,4500.0,25.0,25.0,18916.67,20171229,0.184300
13182217,3017,366600.0,207584,412400.0,95.0,95.0,23622.84,20171229,0.513815
13182218,3019,68300.0,207605,31800.0,45.0,45.0,72137.74,20171229,0.184629
13182219,3020,859100.0,207608,530900.0,235.0,235.0,117084.13,20171229,0.352270
13182220,3021,2530500.0,207609,1584100.0,1124.0,1124.0,215823.71,20171229,0.365807
13182221,3023,76400.0,207611,18600.0,245.0,245.0,19421.30,20171229,0.162557
13182222,3025,69600.0,207649,11400.0,100.0,100.0,14005.38,20171229,0.119993
13182223,3027,1800.0,207653,600.0,90.0,90.0,2977.63,20171229,0.111573
13182224,3028,2800.0,207654,100.0,110.0,110.0,4027.80,20171229,0.014435


In [8]:
import numpy as np


secid_uni = np.unique(df['secid'])
print(len(df['secid']))
print(len(secid_uni), secid_uni)
print(len(df['secid'])/len(secid_uni))

13182265
7580 [  5005   5015   5016 ... 209477 209554 209570]
1739.085092348285


In [3]:
file_id = '14_1NRu-FvYn0Yy9E_Gcbhn_TQLww0Yjq'  #雲端硬碟檔案連結碼d/
downloaded = drive.CreateFile({'id' : file_id})
downloaded.GetContentFile('weeklyskew.csv')
df = pd.DataFrame(pd.read_csv('weeklyskew.csv')) #index_col='Date'
df.info()
print("original data")
print(df)

KeyboardInterrupt: ignored